In [ ]:
#importing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import smart_open

import logging
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  

import multiprocessing
import docx2txt

from gensim.models import Word2Vec
from sklearn.manifold import TSNE

import texthero as hero

In [ ]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model
# fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
sf_new = pd.read_csv("sfw_new.csv")

In [ ]:
sf_new

In [ ]:
sf_sectors = pd.read_csv("sfw_sector.csv")

In [ ]:
sf_sectors

In [ ]:
sf_sectors['Sector'].value_counts()

In [ ]:
X_train_sf, X_test_sf, y_train_sf, y_test_sf = train_test_split(sf_sectors['Skill Title'], 
                                                    sf_sectors['Sector'], 
                                                    test_size = 0.03, 
                                                    shuffle = True, 
                                                    stratify = sf_sectors['Sector'], 
                                                    random_state = 3)

In [ ]:
transformer = Pipeline([('vect', CountVectorizer(max_features = 5000, stop_words = 'english', min_df = 3, max_df = 0.5)),
                ('tfidf', TfidfTransformer()),
                                
                ])
transformer.fit_transform(sf_sectors['Skill Title'])
# X_test_dtm = transformer.transform(cf_model_IT['Job Description'])

In [ ]:
import pickle

pickle.dump(transformer, open("tfidf.pickle", "wb"))

In [ ]:
k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 50)

In [ ]:
param_grid = {'C': [0.1,1, 10], 'gamma': [1,0.1,0.01, 0.001],'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

In [ ]:
model = GridSearchCV(SVC(class_weight='balanced', probability = True),param_grid,refit=True,verbose=2, n_jobs = -1, cv = 3)

for train_indices, test_indices in k_fold.split(sf_sectors['Skill Title'], sf_sectors['Sector']):
    X_train, X_test = X_train_dtm[train_indices], X_train_dtm[test_indices]
    y_train, y_test = sf_sectors['Sector'][train_indices], sf_sectors['Sector'][test_indices]
    model.fit(X_train, y_train)
    print("Best parameters set found on development set:")
    print()
    print(model.best_params_)
    print()

    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()



# y_pred = logreg.predict(X_test_sf)

In [ ]:
pickle.dump(model, open('industry.sav', 'wb'))



In [ ]:
cf_model_IT.iloc[125]

In [ ]:
len(cf_model_IT)

In [ ]:
model.best_params_

In [ ]:
pd.Series(model.predict(X_test_dtm)).value_counts()

In [ ]:
from sklearn import preprocessing

def get_labels(data, column):
    
    le = preprocessing.LabelEncoder()
    for i in range(len(data)):
        data['Sector Label'] = le.fit_transform(data[column])
        values = sf_sectors[['Sector Label', column]].drop_duplicates().sort_values('Sector Label').reset_index().drop(columns = 'index')
        return values



In [ ]:
vectorizer = pickle.load(open("tfidf.pickle", "rb"))
model = pickle.load(open('industry.sav', 'rb'))
nlp = spacy.load("en_core_web_lg")

def retrieve_job_title(jd, vectorizer, model, data, nlp):
    # replace skills here
    transformed_jd = vectorizer.transform(jd)
    probs = model.predict_proba(transformed_jd)
    best_n = np.argsort(probs, axis=1)[:,-3:]
    labels = get_labels(data, 'Sector')
    industries = labels.iloc[best_n[0]]['Sector'].to_list()
    print(industries)
    all_jobs = data[(data['Sector']== industries[0]) | (data['Sector']== industries[1]) | (data['Sector']== industries[2])]
    
    cf = nlp(str(jd))
    print("User Input:")
    print(cf)

    scores = []
    sfw_jobs = []
    for j in range(0, len(all_jobs)):
        sfw = nlp(str(all_jobs.iloc[j]['Skill Title']))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(all_jobs.iloc[j]['Job_Role_Replaced'])
        
    ind1, ind2, ind3, ind4, ind5 = heapq.nlargest(5, zip(scores, sfw_sector_IT['Job_Role_Replaced']))
    print(ind1)
    print(ind2)
    print(ind3)

    
    

In [ ]:
retrieve_job_title(pd.Series(cf_model_IT.iloc[125]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp)

In [ ]:
retrieve_job_title(pd.Series(cf_model_IT.iloc[125]['Job Description']), vectorizer, model, sf_sectors, nlp)

In [ ]:
retrieve_job_title(pd.Series(cf_model.iloc[10]['Skills_unlisted']), vectorizer, model, sf_sectors, nlp)

In [ ]:
model

In [ ]:
model = SVC(C = 1, gamma = 1, kernel = 'linear', class_weight='balanced', probability=True)

In [ ]:
model.fit(X_train_dtm, sf_sectors['Sector'])

In [ ]:
probs = model.predict_proba(transformer.transform(pd.Series(cf_model.iloc[10]['Job Description'])))

In [ ]:
best_n = np.argsort(probs, axis=1)[:,-3:]

In [ ]:
best_n

In [ ]:
# jd = transformer.transform(pd.Series(cf_model.iloc[10]['Skills_unlisted']))

data = sf_sectors
labels = get_labels(sf_sectors,'Sector')
industries = labels.iloc[best_n[0]]['Sector'].to_list()
print(industries)
all_jobs = data[(data['Sector']== industries[0]) | (data['Sector']== industries[1]) | (data['Sector']== industries[2])]

cf = nlp(str(jd))


scores = []
sfw_jobs = []
for j in range(0, len(all_jobs)):
    sfw = nlp(str(all_jobs.iloc[j]['Skill Title']))
    scores.append(sfw.similarity(cf))
    sfw_jobs.append(all_jobs.iloc[j]['Job_Role_Replaced'])

ind1, ind2, ind3, ind4, ind5 = heapq.nlargest(5, zip(scores, all_jobs['Job_Role_Replaced']))
print(ind1)
print(ind2)
print(ind3)

In [ ]:
values

In [ ]:
cf_model_IT.iloc[125]['Job Description']

In [ ]:
model.predict(X_test_dtm[2])

In [ ]:
probs = model.predict_proba(X_test_dtm[2])
best_n = np.argsort(probs, axis=1)[:,-2:]
best_n

In [ ]:
best_n[0]


In [ ]:
values.iloc[best_n[0]]['Sector'].to_list()

In [ ]:
probs = model.predict_proba(X_test_dtm[125])
best_n = np.argsort(probs, axis=1)[:,-2:]
best_n

In [ ]:
!pip install en_core_web_lg

In [ ]:
import nltk

import re
import string

import heapq
import spacy

#nlp = spacy.load("en_core_web_lg")

In [ ]:
sfw_sector_IT = sf_sectors[(sf_sectors['Sector']=='Infocomm Technology') | (sf_sectors['Sector']=='Media')]
sfw_sector_IT

In [ ]:
cf = nlp(str(cf_model_IT.iloc[135]['Skills_unlisted']))
print(cf)
# try with replaced job description
# rerun the whoel thing with job description
scores = []
sfw_jobs = []
for j in range(0, len(sfw_sector_IT)):
    sfw = nlp(str(sfw_sector_IT.iloc[j]['Skill Title']))
    scores.append(sfw.similarity(cf))
    sfw_jobs.append(sfw_sector_IT.iloc[j]['Job_Role_Replaced'])

In [ ]:
ind1, ind2, ind3, ind4, ind5 = heapq.nlargest(5, zip(scores, sfw_sector_IT['Job_Role_Replaced']))

#     print(sfw_sector_IT.iloc[ind1]['Job_Role_Replaced'])
#     closest_2_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind2])
#     closest_3_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind3])
# cf_jobs.append(IT['Job Title'][i])

In [ ]:
ind1

In [ ]:
df_IT_72 = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_3_sfw_jobs':closest_3_sfw_jobs})

In [ ]:
logreg.predict(pd.Series(cf_model_IT.iloc[2]['Job Description']))

In [ ]:
sf_sectors['Sector'].unique()

In [ ]:
cf_model_fin = cf_model[cf_model.Industry_unlisted == "Banking and Finance"]

In [ ]:
cf_model_fin

In [ ]:
pd.Series(model.predict(transformer.transform(cf_model_fin['Job Description']))).value_counts()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 300],
    'class_weight': ['balanced', 'balanced_subsample']
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in range(len(sf_sectors['Sector'])):
    sf_sectors['Sector Label'] = le.fit_transform(sf_sectors['Sector'])

In [ ]:
sf_sectors

In [ ]:
sf_sectors[['Sector', 'Sector Label']].drop_duplicates().sort_values('Sector Label')

In [ ]:
for train_indices, test_indices in k_fold.split(sf_sectors['Skill Title'], sf_sectors['Sector']):
    X_train, X_test = X_train_dtm[train_indices], X_train_dtm[test_indices]
    y_train, y_test = sf_sectors['Sector'][train_indices], sf_sectors['Sector'][test_indices]
    grid_search.fit(X_train, y_train)
    print("Best parameters set found on development set:")
    print()
    print(grid_search.best_params_)
    print()

#     y_true, y_pred = y_test, grid_search.predict(X_test)
#     print(classification_report(y_true, y_pred))
#     print()

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.fit(X_train_dtm, sf_sectors['Sector'])

In [ ]:
grid_predictions = grid_search.predict(X_test_dtm)

In [ ]:
pd.Series(grid_predictions).value_counts()

In [ ]:
probs = grid_search.predict_proba(X_test_dtm[2])
best_n = np.argsort(probs, axis=1)[:,-3:]
best_n

In [ ]:
cf_model_IT.iloc[2]

In [ ]:
pd.Series(model.predict(transformer.transform(cf_model_fin['Job Description']))).value_counts()

In [ ]:
len(cf_model_fin)

In [ ]:
cf_model_fin.iloc[2]['Job Description']

In [ ]:
probs = model.predict_proba(transformer.transform(pd.Series(cf_model_fin.iloc[2]['Job Description'])))
best_n = np.argsort(probs, axis=1)[:,-3:]
best_n

In [ ]:
pd.Series(logreg.predict(cf_model_IT['Job Description'])).value_counts()

In [ ]:
sf_sectors['Sector'].value_counts()

In [ ]:
from gensim.models.phrases import Phrases, Phraser
jobD = [row.split() for row in sf_new['Skill description']]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]

sf_new['bigrams'] = sentences

In [ ]:
jobD = [row.split() for row in sf_new['Skill Title']]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]

sf_new['bigrams_skills'] = sentences

In [ ]:
from Portal import *
# pd.set_option('display.width', 400)
# pd.set_option('display.max_columns', 10)


class CareersFuture(Portal):

    def __init__(self, name):
        super().__init__(name)

    def read_data(self):
        # Reads data
        df = pd.read_csv('mycareerfuture2020-09-15.csv', error_bad_lines=False)
        return df

    def clean_data(self):
        df = self.read_data()
        # df = df[df['Job Description'].str.contains('value')]
        df['Job Description'] = df['Job Description'].astype(str)
        df['Job Description'] = df['Job Description'].apply(lambda x: self.strip_html_tags(x))

        df['Industry'] = df['Industry'].astype(str)
        df['Industry'] = df['Industry'].apply(lambda x: self.str_to_literal(x))

        # df['Skills'] = df['Skills'].astype(str)

        df['Job Experience Required (years)'] = df['Job Experience Required (years)'].astype(int)

        df['Job Monthly Min Sal'] = df['Job Monthly Min Sal'].astype(int)
        df['Job Monthly Max Sal'] = df['Job Monthly Max Sal'].astype(int)

        return df

    def str_to_literal(self, text):
        try:
            # ast.literal_eval function converts takes in a string and converts it into a dictionary
            ls = super().str_to_literal(text)
            industry = []
            for i in ls:
                industry.append(i['category'])
            return industry
        except:
            pass

careersfuture = CareersFuture("careersfuture")


In [ ]:
cf = pd.DataFrame(careersfuture.clean_data())
cf['Industry_unlisted'] = cf['Industry'].apply(lambda x: ','.join(map(str, x)))
cf['Skills'] = cf['Skills'].apply(lambda x: x.replace("'", "") )
cf['Skills_unlisted'] = cf['Skills'].apply(lambda x: x.replace('[','').replace(']',''))
cf_model = cf[["Job Title", "Job Description", "Industry_unlisted", "Skills_unlisted", "Job Experience Required (years)","Job Monthly Min Sal", "Job Monthly Max Sal" ]]
cf_model_IT = cf_model[cf_model['Industry_unlisted']=="Information Technology"]


In [ ]:
cf

In [ ]:
X_test = cf_model_IT['Job Description']

In [ ]:
cf_model_IT.iloc[12]

In [ ]:
sf_new['bigrams'] = sf_new['bigrams'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
sf_new['bigrams_skills'] = sf_new['bigrams_skills'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sf_new["bigrams_skills"])
Y = vectorizer.transform(X_test)


word_features = vectorizer.get_feature_names()
word_features[100:200]

In [ ]:
kmeans = KMeans(n_clusters = 20, n_init = 5, n_jobs = -1)
kmeans.fit(X)

In [ ]:
sf_new['Clusters'] = kmeans.labels_

In [ ]:
sf_new

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(word_features[word] for word in centroid))

In [ ]:
import seaborn as sns

clusters = sf_new[sf_new.Job_Role_Replaced.str.contains("health")].groupby(['Clusters', 'Job_Role_Replaced']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'Job_Role_Replaced'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('Job_Role_Replaced', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('Clusters', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")

In [ ]:
import seaborn as sns

clusters = sf_new[sf_new.Job_Role_Replaced.str.contains("air")].groupby(['Clusters', 'Job_Role_Replaced']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'Job_Role_Replaced'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('Job_Role_Replaced', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('Clusters', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")

In [ ]:
sf_new.Clusters.value_counts()

In [ ]:
sf_new.to_csv("sf_clusters.csv")

In [ ]:
X_train_sf, X_test_sf, y_train_sf, y_test_sf = train_test_split(sf_new['Skill description'], 
                                                    sf_new['Clusters'], 
                                                    test_size = .3, 
                                                    shuffle = True, 
                                                    stratify = sf_new['Clusters'], 
                                                    random_state = 3)

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(X_train_sf, y_train_sf)


y_pred = logreg.predict(X_test_sf)

In [ ]:
accuracy_score(y_test_sf, y_pred)

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(sf_new['Skill description'], sf_new['Clusters'])


y_pred = logreg.predict(cf_model_IT['Job Description'])

In [ ]:
pd.Series(y_pred).value_counts()

In [ ]:
pd.Series(kmeans.predict(Y)).value_counts()

In [ ]:
cf_model_IT.iloc[6]

In [ ]:
cf_model_IT.iloc[6]['Job Description']

In [ ]:
kmeans.predict(Y[6])

In [ ]:
logreg.predict(pd.Series(cf_model_IT.iloc[6]['Job Description']))

In [ ]:
sf_new[sf_new['Job_Role_Replaced']=='ui design']

the kmeans and log reg predict wrongly! 
For software development/IT, it should come under cluster 13 (eyeballing the clusters above)
13 : business, data, management, security, performance, processes, stakeholders, cyber, requirements, network

In [ ]:
sf_new[sf_new.Clusters==7]['Job_Role_Replaced'].unique()

In [ ]:
documents = sf_new[sf_new.Clusters == 7]['Skill description']

In [ ]:
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.

sentences = [dictionary.doc2bow(simple_preprocess(sent)) for sent in documents]



In [ ]:
jd_1 = dictionary.doc2bow(simple_preprocess(X_test.iloc[6]))

In [ ]:
maximum = 0
index = 0
second_maximum = 0
second_index = 0
for i in range(len(sentences)):

    if softcossim(jd_1, sentences[i], similarity_matrix)> maximum:
        second_maximum = maximum
        second_index = index
        maximum = softcossim(jd_1, sentences[i], similarity_matrix)
        index = i
        
    
print(maximum)
print(index)
    
print(second_maximum)
print(second_index)

In [ ]:
sf_new[sf_new.Clusters == 7].iloc[115]['Job_Role_Replaced']

In [ ]:
sf_new[sf_new.Clusters == 7].iloc[56]['Job_Role_Replaced']

In [ ]:
sf_new[sf_new.Clusters == 7].iloc[49]['Skill description']

In [ ]:
logreg = Pipeline([('vect', CountVectorizer(stopwords='english', max_df = 0.5)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier()),
                
               ])
logreg.fit(sf_new[sf_new.Clusters == 13]['Skill description'], sf_new[sf_new.Clusters == 13]['Job_Role_Replaced'])


y_pred = logreg.predict(pd.Series(cf_model_IT.iloc[6]['Job Description']))
y_pred

Others

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [ ]:
pd.Series([x.strip(' ') for x in sf_model_IT[sf_model_IT['cluster']==3]['Job_Role_Replaced']]).unique()

Unnecessay stuff

In [ ]:
number_clusters = range(1, 20)

kmeans = [KMeans(n_clusters=i, max_iter = 600) for i in number_clusters]
kmeans

score = [kmeans[i].fit(Y_sklearn).score(Y_sklearn) for i in range(len(kmeans))]
score

plt.plot(number_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Method')
plt.show()

In [ ]:
results1 = pd.concat([pd.Series(X_test_sf).reset_index(), pd.Series(y_test_sf).reset_index(), pd.Series(y_pred).reset_index()], axis = 1)